In [94]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
import pandas as pd
import numpy as np
import lasio
import os
from pyspark.sql.types import *
from scipy import interpolate
import itertools
import operator
from operator import itemgetter
from dtw import dtw

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import mean as _mean, col,column
from pyspark.sql.functions import max, min
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [4]:
from pyspark.sql.functions import length
targ_form = spark.read.option('header','True').csv('C://Users/Сергей/Desktop/HW/FieldWTPS1.csv',sep=';')
targ_form = targ_form.groupby(['Well','Surface']).agg({'MD':'mean'}).withColumnRenamed("avg(MD)", "MD")
files=[]
length=targ_form.select("MD").count()
for i in range(length):
    files.append( targ_form.collect()[i][0])
files = set(files)

In [8]:
for i_file in files:
    try:
        mySchema = StructType([ StructField("DEPT", DoubleType(), True),StructField("SP", DoubleType(), True)])
        cor = lasio.read('C://Users/Сергей/Desktop/HW/path/' + i_file +'.las')
        SP = pd.DataFrame({'DEPT':cor['DEPT'], 'SP':cor['SP']})
        SP = spark.createDataFrame(SP,schema=mySchema)
        SP = SP.fillna(0)  
        SP = SP.where((col('SP') != 0) & (col("SP") != 9999) & (col("SP") != (-9999)))
        if targ_form.where(col("Well")==i_file.split('.')[0]).distinct().count()==2:
            top=targ_form.select("MD").where((col("Well")==i_file.split('.')[0])&(col("Surface")=='Top_Georg')).collect()[0][0]
            bottom = targ_form.select("MD").where((col("Well")==i_file.split('.')[0])&(col("Surface")=='U1-3_bot')).collect()[0][0]
        else:
            top=targ_form.select("MD").where((col("Well")==i_file.split('.')[0])&(col("Surface")=='Top_Georg')).collect()[0][0]
            bottom = top+100
        SP=SP.where((col("DEPT") >= top)  & (col("DEPT") <= bottom))
        if SP.count()>0:
            typic = SP.select(col("DEPT").alias("Y"),col("SP").alias("X"))
            minimum_Y=typic.select(min(col('Y'))).collect()[0][0]
            maximum_Y=typic.select(max(col('Y'))).collect()[0][0]
            typic_Y=typic.select(((col('Y')-minimum_Y)*100/(maximum_Y-minimum_Y)).alias('Y'))
            array_Y = np.asarray(typic_Y.select("*").rdd.flatMap(lambda x: x).collect())
            minimum_X=typic.select(min(col('X'))).collect()[0][0]
            maximum_X=typic.select(max(col('X'))).collect()[0][0]
            typic_X=typic.select(((col('X')-minimum_X)*1/(maximum_X-minimum_X)).alias('X'))
            array_X = np.asarray(typic_X.select("*").rdd.flatMap(lambda x: x).collect())
            f = interpolate.interp1d(array_Y, array_X)
            ynew = np.arange(0,100,1)
            xnew = f(ynew)
            corotazh_total = pd.DataFrame({'Y':ynew[::-1],'X':xnew})
            corotazh_total.to_csv('C://Users/Сергей/Desktop/HW/OutData2/'+i_file.split('.')[0]+'.csv')
    except Exception:
        pass

Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.
Header section Parameter regexp=~P was not found.


In [10]:
files = os.listdir(path ="C://Users/Сергей/Desktop/HW/typical/CSV")
for i in files:
    mySchema_2 = StructType([ StructField("X", DoubleType(), True),StructField("Y", DoubleType(), True)])
    typic = spark.read.option('header','True').csv('C://Users/Сергей/Desktop/HW/typical/CSV/' + i, sep=';',schema=mySchema_2)
    minimum_Y=typic.select(min(col('Y'))).collect()[0][0]
    maximum_Y=typic.select(max(col('Y'))).collect()[0][0]
    typic_Y=typic.select(((col('Y')-minimum_Y)*55/(maximum_Y-minimum_Y)).alias('Y'))
    array_Y = np.asarray(typic_Y.select("*").rdd.flatMap(lambda x: x).collect())
    minimum_X=typic.select(min(col('X'))).collect()[0][0]
    maximum_X=typic.select(max(col('X'))).collect()[0][0]
    typic_X=typic.select(((col('X')-minimum_X)*1/(maximum_X-minimum_X)).alias('X'))
    array_X = np.asarray(typic_X.select("*").rdd.flatMap(lambda x: x).collect())
    f = interpolate.interp1d(array_Y, array_X)
    xnew = f(np.arange(0,55,1))
    corotazh_total = pd.DataFrame({'Y':np.arange(0,55,1),'X':xnew})
    corotazh_total.to_csv('C://Users/Сергей/Desktop/HW/typical/reb/' + i)

In [22]:
class my_dictionary(dict):
    def __init__(self):
        self = dict()
    def add(self, key, value):
        self[key] = value

In [131]:
files = os.listdir(path ="C://Users/Сергей/Desktop/HW/OutData2")
mySchema_3 = StructType([ StructField("well", StringType(), True),
                         StructField("typic", StringType(), True),
                         StructField("value", DoubleType(), True)])
data = []
data_1 = spark.createDataFrame(data,schema=mySchema_3)
g = ['1','2','3','4','5','6','7','8','9','10']
for u in files:
    my_dict = my_dictionary()
    SP = spark.read.option('header','True').csv('C://Users/Сергей/Desktop/HW/OutData2/'+u)
    SP = SP.select(col("Y").cast(DoubleType()),col("X").cast(DoubleType()))
    x = SP.select(col('X'))
    array_x = np.asarray(x.select("*").rdd.flatMap(lambda x: x).collect()).tolist()
    for i in g:
        SP1 = spark.read.option('header','True').csv('C://Users/Сергей/Desktop/HW/typical/reb/'+i+'.csv')
        SP1 = SP1.select(col("Y").cast(DoubleType()),col("X").cast(DoubleType()))
        SP1.select(col('Y'),col("X"))
        y = SP1.select(col("X"))
        array_y=np.asarray(y.select("*").rdd.flatMap(lambda x: x).collect()).tolist()
        manhattan_distance = lambda x, y: np.abs(x - y)
        d, cost_matrix, acc_cost_matrix, path = dtw(array_x, array_y, dist = manhattan_distance)
        my_dict.add(i,d)
    for key, value in itertools.islice(sorted( my_dict.items(), key =  operator.itemgetter(1)),1):
        mySchema_4 = StructType([ StructField("well", StringType(), True),
                         StructField("typic", StringType(), True),
                         StructField("value", DoubleType(), True)])
        data_1 = data_1.unionByName(spark.createDataFrame(pd.DataFrame({'well':[u],"typic":[key],"value":[value]}),mySchema_4))
data_1.toPandas().to_csv('C://Users/Сергей/Desktop/HW/mark.csv')